In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from tensorflow.keras.metrics import Recall, Precision
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import Sequential, layers, models
import tensorflow as tf
import matplotlib as plt

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0/255.0,
    dtype='float32'
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    dtype='float32'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    dtype='float32'
)

In [ ]:
test_generator = tf.keras.utils.image_dataset_from_directory("gs://pneumonia_diagnosis/original_data/test",
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42)

train_generator = tf.keras.utils.image_dataset_from_directory("gs://pneumonia_diagnosis/original_data/train",
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42)

val_generator = tf.keras.utils.image_dataset_from_directory("gs://pneumonia_diagnosis/original_data/val",
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42)

In [ ]:
def compile(model):
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', Recall()])
    print(model.summary())
    return model

In [ ]:
def fitting(model):
    es = EarlyStopping(patience = 10, monitor = 'val_loss')
    model.fit(train_generator, epochs = 5, verbose = True, use_multiprocessing=True, validation_data = val_generator, callbacks = [es])

In [ ]:
def dense_net():
    model = DenseNet121(input_shape=(256, 256, 3), include_top=False, weights='imagenet', pooling='avg')
    model.trainable = False
    
    
    x = layers.Flatten()(model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(model.input, x)
    
    return model

In [ ]:
dense_model = dense_net()

In [ ]:
compile(dense_model)
dense_model_history = fitting(dense_model)

In [ ]:
# 

plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(dense_model_history.history['loss'], label='Loss')
plt.plot(dense_model_history.history['val_loss'], label='Val_Loss')
plt.ylim(0, 0.3)
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(dense_model_history.history['accuracy'], label='Accuracy')
plt.plot(dense_model_history.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')